In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier, MLPRegressor
from lightgbm import LGBMRegressor,LGBMClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV

C:\Users\wj\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data_x = pd.read_csv("./data/practice/acic_practice_0001.csv")
data_y = pd.read_csv("./data/practice_year/acic_practice_year_0001.csv")

# propensity score

In [56]:
data_x

,id.practice,X1,X2,X3,X4,X5,X6,X7,X8,X9
0,1,0,A,1,A,1,20.774076,14.153255,0.161126,43.431874
1,2,0,A,0,C,0,33.565928,3.284657,0.556784,12.721988
2,3,0,C,1,A,1,57.283021,11.178051,0.257244,-7.352617
3,4,1,C,1,A,0,41.900776,1.542463,0.129618,20.032199
4,5,1,B,1,A,0,41.486809,7.504068,0.413924,35.449536
...,...,...,...,...,...,...,...,...,...,...
495,496,0,C,1,A,1,29.898841,17.945129,0.381701,40.224974
496,497,1,C,0,A,1,20.378333,20.770890,0.422255,62.857150
497,498,0,A,0,A,0,30.162733,9.174455,0.205427,-4.931348
498,499,1,C,0,A,1,32.795499,19.769398,0.295538,84.228736


In [49]:
data_z = pd.merge(data_x, data_y[['id.practice','Z']].drop_duplicates(), on='id.practice')
df = data_z
ct = ['X2','X4']
for c in ct:
    df[c] = df[c].astype('category')

debias_m = LGBMClassifier(max_depth=3)

X = ['X1','X2','X3','X4','X5','X6','X7','X8','X9']
T = ['Z']
#ps_res =  df[T] - cross_val_predict(debias_m, df[X], df[T].values.ravel(), cv=5, method='predict_proba')[:,1].reshape(-1,1) + df[T].mean()

In [57]:
debias_m = LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         #metric = 'log_loss',
                         metric = 'binary_logloss,auc',
                         max_depth = 4,
                         num_leaves = 20,
                         learning_rate = 0.1,
                         #feature_fraction = 0.7,
                         min_child_samples=21,
                         min_child_weight=0.001,
                         #bagging = 1,
                         #subsample_freq = 2,
                         reg_alpha = 0.002,
                         reg_lambda = 10,
                         cat_smooth = 0,
                         n_estimators = 200,   
                        )

ps = cross_val_predict(debias_m, df[X], df[T].values.ravel(), cv=10, method='predict_proba')[:,1]
#ps_res =  df[T] - cross_val_predict(debias_m, df[X], df[T].values.ravel(), cv=5, method='predict_proba')[:,1].reshape(-1,1) + df[T].mean()
data_x = data_x.assign(ps = ps)

In [ ]:
data_z = data_z.assign(ps = ps)
data_z

# average outcome for year 3,4

In [67]:
data_o = pd.merge(data_x, data_y, on='id.practice')
df = data_o
ct = ['X2','X4']
for c in ct:
    df[c] = df[c].astype('category')
    
#XV = df.columns.drop(['id.practice','year','post','Z','n.patients'])
V = ['V1_avg', 'V2_avg', 'V3_avg', 'V4_avg', 'V5_A_avg', 'V5_B_avg', 'V5_C_avg']
Y = ['Y']

In [68]:
means = []
for i in range(1,5):
    means.append(df[(df['year']==i) & (df['Z']==0)]['Y'].mean())
    print(means[i-1])

avgs = pd.DataFrame({'year': [1, 2, 3, 4], 't_avg': means})
df = pd.merge(df, avgs, on='year')

df_34 = df[df['year'] >= 3]

870.6451336763548
1010.0760840822331
1134.3928342316492
1250.6285559851644


In [69]:
denoise_m = LGBMRegressor(is_unbalance = True,
                         #metric = 'log_loss',
                         metric = 'binary_logloss,auc',
                         max_depth = 4,
                         num_leaves = 20,
                         learning_rate = 0.1,
                         #feature_fraction = 0.7,
                         min_child_samples=21,
                         min_child_weight=0.001,
                         #bagging = 1,
                         #subsample_freq = 2,
                         reg_alpha = 0.002,
                         reg_lambda = 10,
                         cat_smooth = 0,
                         n_estimators = 200,   
                        )

# \hat{\mu_0(v)+e(x)r(x)+f(t)}
#pred3 = cross_val_predict(denoise_m, df_3[V], df_3[Y].values.ravel(), cv=10, method='predict_proba')[:,1]
#pred4 = cross_val_predict(denoise_m, df_4[V], df_4[Y].values.ravel(), cv=10, method='predict_proba')[:,1]

df_34 = df[df['year'] >= 3]
pred_M = cross_val_predict(denoise_m, df_34[V], df_34['Y']-df_34['t_avg'], cv=10)

# Non Parametric Double/Debiased ML

In [77]:
df_34 = df_34.assign(y_res = df_34['Y'] - pred_M, z_res = df_34['Z'] - df_34['ps'])

model_final = LGBMRegressor(max_depth=6)
 
# create the weights
w = df_34['z_res'] ** 2 
 
# create the transformed target
y_star = (df_34['y_res'] / df_34['z_res'])
 
# use a weighted regression ML model to predict the target with the weights.
model_final.fit(X=df_34[V], y=y_star, sample_weight=w)

LGBMRegressor(max_depth=6)

# Iterative Estimation